# Using the PyMISP objects
## MISPEvent

In [ ]:
from pymisp import MISPEvent

event = MISPEvent()

event.info = 'This is my new MISP event'  # Required
event.distribution = 0  # Optional, defaults to MISP.default_event_distribution in MISP config
event.threat_level_id = 2  # Optional, defaults to MISP.default_event_threat_level in MISP config
event.analysis = 1  # Optional, defaults to 0 (initial analysis)

print(event.to_json())

## Tag Event

In [ ]:
event.add_tag('tlp:white')

print(event.to_json())

## Set the Event date

In [ ]:
# As text
event.set_date('2018-04-13')
print('Simple', event.date)
event.set_date('Sat Oct 11 00:13:46 2017')
print('Mess', event.date)

from datetime import date
d = date.today()
print(type(d))
event.set_date(d)
print(event.date)


from datetime import datetime
d = datetime.now()
print(type(d))
event.set_date(d)
print(event.date)

## Add Attribute to event

In [ ]:
attribute = event.add_attribute('ip-dst', '8.8.8.8')  # Minimal parameters

print(type(attribute))

print(attribute.to_json())

## Modify existing attribute

In [ ]:
attribute.to_ids = False

print(attribute.to_json())

## Set parameters (inline)

In [ ]:
attribute_second = event.add_attribute('ip-dst', '8.8.8.9', disable_correlation=True)  # Minimal parameters

print(attribute_second.to_json())

## Tag Attribute

In [ ]:
attribute_second.add_tag('tlp:amber')

print(attribute_second.to_json())

## Soft delete attribute

In [ ]:
attribute.delete()
print(attribute.to_json())

## MISPObject

In [ ]:
from pymisp import MISPObject

circl_attr = event.add_attribute('ip-dst', '149.13.33.14')


misp_object = MISPObject('domain-ip', standalone=False, default_attributes_parameters=circl_attr)
# Notes: 
# * standalone: this object will be attached to a MISPEvent, so the references will be in the dump
# * default_attributes_parameters: keep parameters from a MISPAttribute (usefull when expanding a existing one) 
misp_object.comment = 'My Fancy new object'

obj_attr = misp_object.add_attribute('domain', value='circl.lu')
obj_attr.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14')
misp_object.add_attribute('first-seen', value='2018-04-11')
misp_object.add_attribute('last-seen', value='2018-06-11')
misp_object.add_reference(circl_attr.uuid, 'related-to', 'Expanded with passive DNS entry')

event.add_object(misp_object)
print(event.to_json())


## Helpers for MISPObjects 

In [ ]:
from pymisp.tools import FileObject

file_obj = FileObject(filepath='../../tests/viper-test-files/test_files/EICAR.com', standalone=False)
print(file_obj.to_json())

In [ ]:
event.add_object(file_obj)
print(event.to_json())

## Playing with a malware sample

In [ ]:
sample = file_obj.get_attributes_by_relation('malware-sample')[0]
print(sample)
print(sample.malware_binary)
print(sample.malware_binary.getvalue())

## Use lief to extract indicators out of binaries

In [ ]:
from pymisp.tools import make_binary_objects

file_obj, bin_obj, sections = make_binary_objects(filepath='../../tests/viper-test-files/test_files/whoami.exe', standalone=False)

event.add_object(file_obj)
event.add_object(bin_obj)
for s in sections:
    event.add_object(s)

## References

In [ ]:
print(bin_obj.uuid)
print(bin_obj.references[0].to_json())

## Change creator

In [ ]:
from pymisp import MISPOrganisation
orgc = MISPOrganisation()
orgc.name = 'bazbaz'
orgc.id = 15
orgc.uuid = '5888a98d-a7e8-4183-94bb-4d19950d210f'
# NOTE: Pushing this object will only work if the user has sync right (if not, the orgc key will be ignored)
event.Orgc = orgc

## Mark event as published

In [ ]:
event.publish()
print(event.published)

## Dump valid MISP Event ready to push to MISP

In [ ]:
print(event.to_json())

# Update an existing MISPEvent

In [ ]:
from pymisp import MISPEvent

existing_event = MISPEvent()
existing_event.load_file('../../tests/mispevent_testfiles/existing_event.json')

print(existing_event.attributes[0])
print(existing_event.attributes[0].tags)
print(existing_event.attributes[0].timestamp)
print(existing_event.attributes[0].to_json())

## Edit, removes the timestamp when exporting

In [ ]:
existing_event.attributes[0].add_tag('tlp:white')
print(existing_event.attributes[0].to_json())